<a href="https://colab.research.google.com/github/ariahosseini/DeepML/blob/main/009_TensorFlow_Proj_Nine_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, warnings, pathlib, shutil, random

In [ ]:
warnings.filterwarnings(action="ignore")

In [ ]:
print("tensorflow:", tf.__version__)

tensorflow: 2.13.0


In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  51.0M      0  0:00:01  0:00:01 --:--:-- 51.0M


In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
!cat aclImdb/train/pos/10001_10.txt

Brilliant over-acting by Lesley Ann Warren. Best dramatic hobo lady I have ever seen, and love scenes in clothes warehouse are second to none. The corn on face is a classic, as good as anything in Blazing Saddles. The take on lawyers is also superb. After being accused of being a turncoat, selling out his boss, and being dishonest the lawyer of Pepto Bolt shrugs indifferently "I'm a lawyer" he says. Three funny words. Jeffrey Tambor, a favorite from the later Larry Sanders show, is fantastic here too as a mad millionaire who wants to crush the ghetto. His character is more malevolent than usual. The hospital scene, and the scene where the homeless invade a demolition site, are all-time classics. Look for the legs scene and the two big diggers fighting (one bleeds). This movie gets better each time I see it (which is quite often).

In [ ]:
base_dir = pathlib.Path("aclImdb") # define the directories that will be used to store the data.
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg", "pos"):
    os.makedirs(val_dir / category) # Creates a subdirectory in val_dir with the name of the category (e.g. "val/neg").
    files = os.listdir(train_dir / category) # Lists all the files in the corresponding category directory in train_dir.
    random.Random(1337).shuffle(files) # Shuffles the list of files in a reproducible way using a fixed seed (1337).
    num_val_samples = int(0.2 * len(files)) # Calculates the number of validation samples as 20% of the total number of files in the category.
    val_files = files[-num_val_samples:] # Selects the last num_val_samples files in the shuffled list as the validation files.
    for fname in val_files: # Moves each validation file from train_dir to val_dir for the current category using shutil.move()
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)


In [ ]:
batch_size = 32
train_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size)
val_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size)
test_ds = tf.keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds.take(1):
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"The Gospel of Lou was a major disappointment for me. I had received an E-Mail from the theater showing it that it was a great and inspirational movie. It was neither great nor inspirational. The cinematography was pretty iffy with the whole movie. A lot the scenes were flash backs that were done in a way that couldn't tell at times what they were about. The voices were often distorted for no reason. Also many of the people in the movie were far fetched. The relationship he has with his ex & son is never made clear. Also the whole movie has most him one way, and then all of a sudden BAM, he is cured and inspiring people. The whole movie seems to show that boxing is one of the things that is bad in his life, making him live his life the way that he is living it, but when he changes, he doesn't leave boxing, he teaches others how to box. Thumbs Down.", shape=(), 

In [ ]:
text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",
) # This layer will be used to vectorize the textual data in the datasets.

# prepare the TextVectorization layer for use by fitting it to the training dataset.
text_only_train_ds = train_ds.map(lambda x,y : x) # discarding the labels y.
text_vectorization.adapt(text_only_train_ds) #  adapt method analyzes the text data to generate a vocabulary and configure the layer's settings for text processing.

# creating new datasets with the vectorized data
binary_1gram_train_ds = train_ds.map(
    lambda x,y : (text_vectorization(x), y),
    num_parallel_calls=4) #  The num_parallel_calls argument specifies the number of parallel calls to use for the mapping operation, which can speed up processing for large datasets.
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [ ]:
for inputs, targets in binary_1gram_train_ds.take(1):
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([0. 1. 0. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
def get_model(max_tokens=20000, hidden_dim=16):
    inputs = tf.keras.Input(shape=(max_tokens,))
    x = tf.keras.layers.Dense(hidden_dim, activation="relu")(inputs)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
model = get_model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10)

Epoch 1/10
625/625 [==============================] - 9s 14ms/step - loss: 0.4014 - accuracy: 0.8296 - val_loss: 0.2808 - val_accuracy: 0.8874
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2682 - accuracy: 0.9009 - val_loss: 0.2789 - val_accuracy: 0.8944
Epoch 3/10
625/625 [==============================] - 4s 6ms/step - loss: 0.2421 - accuracy: 0.9128 - val_loss: 0.2930 - val_accuracy: 0.8960
Epoch 4/10
625/625 [==============================] - 3s 6ms/step - loss: 0.2229 - accuracy: 0.9259 - val_loss: 0.3033 - val_accuracy: 0.8958
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2148 - accuracy: 0.9303 - val_loss: 0.3199 - val_accuracy: 0.8978
Epoch 6/10
625/625 [==============================] - 4s 7ms/step - loss: 0.2132 - accuracy: 0.9319 - val_loss: 0.3268 - val_accuracy: 0.8960
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2169 - accuracy: 0.9309 - val_loss: 0.3425 - val_accuracy: 0.8950
Epoch

In [ ]:
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 7s 8ms/step - loss: 0.3764 - accuracy: 0.8824
Test acc: 0.882


In [ ]:
text_vectorization = tf.keras.layers.TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
model = get_model()
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=3)
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Epoch 1/3
625/625 [==============================] - 11s 16ms/step - loss: 0.3790 - accuracy: 0.8456 - val_loss: 0.2752 - val_accuracy: 0.8926
Epoch 2/3
625/625 [==============================] - 4s 6ms/step - loss: 0.2360 - accuracy: 0.9136 - val_loss: 0.2682 - val_accuracy: 0.9010
Epoch 3/3
782/782 [==============================] - 5s 6ms/step - loss: 0.2952 - accuracy: 0.8927
Test acc: 0.893


In [ ]:
text_vectorization = tf.keras.layers.TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="tf_idf",
)
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
model = get_model()
model.fit(binary_2gram_train_ds.cache(),
          validation_data=binary_2gram_val_ds.cache(),
          epochs=3)
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Epoch 1/3
625/625 [==============================] - 10s 16ms/step - loss: 0.5257 - accuracy: 0.7534 - val_loss: 0.3004 - val_accuracy: 0.8950
Epoch 2/3
625/625 [==============================] - 3s 5ms/step - loss: 0.3368 - accuracy: 0.8620 - val_loss: 0.2918 - val_accuracy: 0.8940
Epoch 3/3
782/782 [==============================] - 5s 6ms/step - loss: 0.3152 - accuracy: 0.8696
Test acc: 0.870


In [ ]:
max_length = 600
max_tokens = 20000
text_vectorization = tf.keras.layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)
int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
embedding_layer = tf.keras.layers.Embedding(input_dim=max_tokens, output_dim=256)
inputs = tf.keras.Input(shape=(None,), dtype="int64")
embedded = tf.keras.layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(embedded)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
inputs = tf.keras.Input(shape=(None,), dtype="int64")
embedded = tf.keras.layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(embedded)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_7 (Dropout)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

In [ ]:
path_to_glove_file = "glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1) #  The maxsplit=1 argument ensures that the split is only performed once, so that the coefficients string can contain spaces.
        coefs = np.fromstring(coefs, "f", sep=" ") # This line converts the string of coefficients to a NumPy array of float32 values. values should be separated by spaces.
        embeddings_index[word] = coefs
print(f"Found {len(embeddings_index)} word vectors.")
# the GloVe file contains pre-trained word vectors for 400,000 unique words.
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary)))) # maps each token in the vocabulary to an integer index.

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
embedding_layer = tf.keras.layers.Embedding(
    max_tokens, # The maximum vocabulary size, which was previously defined as 20,000
    embedding_dim, # The dimensionality of the word embeddings, which was previously defined as 100.
    embeddings_initializer = tf.keras.initializers.Constant(embedding_matrix), # used to initialize the weights with the pre-trained GloVe word embeddings that were loaded earlier.
    trainable=False, # specifies whether the weights of the embedding layer should be trainable during training of the model
    mask_zero=True, # specifies whether the embedding layer should treat zeros as a special "padding" value that should be masked out during training.
)
inputs = tf.keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(embedded)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")
text_vectorization = tf.keras.layers.TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)
# Adapt the TextVectorization layer to the training data
text_vectorization.adapt(text_only_train_ds)
input_text = ["That was an excellent movie, I loved it.", "This was the worst movie ever. I hated it!!"] # list of your new reviews
vectorized_text = text_vectorization(input_text)
output = model.predict(vectorized_text)
print(output)